In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

import gym
import numpy as np
from tensorboardX import SummaryWriter
from collections import namedtuple

In [18]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [19]:
class SkyNet(nn.Module):
    
    def __init__(self, obs_size, hidden_size, n_act):
        super(SkyNet, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_act)
        )

    def forward(self, x):
        return self.net(x)

In [20]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [21]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)

    while True:
        obs_v = torch.Tensor([obs])
        act_prob_v = sm(net(obs_v))
        act_prob = act_prob_v.data.numpy()[0]

        action = np.random.choice(len(act_prob), p=act_prob)
        next_obs, reward, is_done, _ = env.step(action)

        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)

        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [22]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []

    # batch: rwd & steps
    # steps: obs & act


    for reward, steps in batch:
        if reward < reward_bound:
            continue;
        train_obs.extend(map(lambda s: s.observation, steps))
        train_act.extend(map(lambda s: s.action, steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)

    return train_obs_v, train_act_v, reward_bound, reward_mean    

In [23]:
if __name__ == "__main__":
    env = gym.make("CartPole-v0")
    env = gym.wrappers.Monitor(env, directory="mon", force=True)
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = SkyNet(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-cartpole")

    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)

        if reward_m > 199:
            print("Solved!")
            break
    writer.close()

0: loss=0.697, reward_mean=20.9, rw_bound=25.5
1: loss=0.696, reward_mean=19.4, rw_bound=21.5
2: loss=0.680, reward_mean=23.6, rw_bound=24.0
3: loss=0.679, reward_mean=22.4, rw_bound=26.5
4: loss=0.667, reward_mean=31.1, rw_bound=42.5
5: loss=0.660, reward_mean=22.9, rw_bound=23.5
6: loss=0.655, reward_mean=41.1, rw_bound=51.0
7: loss=0.632, reward_mean=36.3, rw_bound=43.5
8: loss=0.621, reward_mean=40.5, rw_bound=49.0
9: loss=0.625, reward_mean=40.4, rw_bound=46.0
10: loss=0.631, reward_mean=44.6, rw_bound=48.5
11: loss=0.603, reward_mean=36.1, rw_bound=47.0
12: loss=0.645, reward_mean=40.1, rw_bound=47.0
13: loss=0.599, reward_mean=51.0, rw_bound=56.0
14: loss=0.582, reward_mean=49.1, rw_bound=58.0
15: loss=0.596, reward_mean=53.2, rw_bound=68.0
16: loss=0.571, reward_mean=64.9, rw_bound=77.0
17: loss=0.584, reward_mean=60.8, rw_bound=61.0
18: loss=0.579, reward_mean=73.6, rw_bound=94.5
19: loss=0.569, reward_mean=81.4, rw_bound=97.0
20: loss=0.578, reward_mean=69.6, rw_bound=82.5
21